# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ffa38e259d0>
├── 'a' --> tensor([[-0.1960,  0.2083,  1.5424],
│                   [-0.1976, -1.3767,  0.1774]])
└── 'x' --> <FastTreeValue 0x7ffa38ebac70>
    └── 'c' --> tensor([[ 0.9793,  0.3693, -0.3448,  0.7939],
                        [-0.7109, -0.5054,  0.7105, -0.4517],
                        [-1.5664, -1.1374,  0.7007,  0.3352]])

In [4]:
t.a

tensor([[-0.1960,  0.2083,  1.5424],
        [-0.1976, -1.3767,  0.1774]])

In [5]:
%timeit t.a

81.9 ns ± 2.05 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ffa38e259d0>
├── 'a' --> tensor([[ 0.4051, -0.8287,  0.8245],
│                   [-0.5784, -0.1190,  1.6041]])
└── 'x' --> <FastTreeValue 0x7ffa38ebac70>
    └── 'c' --> tensor([[ 0.9793,  0.3693, -0.3448,  0.7939],
                        [-0.7109, -0.5054,  0.7105, -0.4517],
                        [-1.5664, -1.1374,  0.7007,  0.3352]])

In [7]:
%timeit t.a = new_value

82.3 ns ± 1.01 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1960,  0.2083,  1.5424],
               [-0.1976, -1.3767,  0.1774]]),
    x: Batch(
           c: tensor([[ 0.9793,  0.3693, -0.3448,  0.7939],
                      [-0.7109, -0.5054,  0.7105, -0.4517],
                      [-1.5664, -1.1374,  0.7007,  0.3352]]),
       ),
)

In [10]:
b.a

tensor([[-0.1960,  0.2083,  1.5424],
        [-0.1976, -1.3767,  0.1774]])

In [11]:
%timeit b.a

73.3 ns ± 0.724 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.2604, -2.5086, -0.5025],
               [-1.6990,  0.5346, -0.1405]]),
    x: Batch(
           c: tensor([[ 0.9793,  0.3693, -0.3448,  0.7939],
                      [-0.7109, -0.5054,  0.7105, -0.4517],
                      [-1.5664, -1.1374,  0.7007,  0.3352]]),
       ),
)

In [13]:
%timeit b.a = new_value

640 ns ± 8.12 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.05 µs ± 10.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.6 µs ± 127 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

298 µs ± 6.68 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

278 µs ± 12 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff98115cb50>
├── 'a' --> tensor([[[-0.1960,  0.2083,  1.5424],
│                    [-0.1976, -1.3767,  0.1774]],
│           
│                   [[-0.1960,  0.2083,  1.5424],
│                    [-0.1976, -1.3767,  0.1774]],
│           
│                   [[-0.1960,  0.2083,  1.5424],
│                    [-0.1976, -1.3767,  0.1774]],
│           
│                   [[-0.1960,  0.2083,  1.5424],
│                    [-0.1976, -1.3767,  0.1774]],
│           
│                   [[-0.1960,  0.2083,  1.5424],
│                    [-0.1976, -1.3767,  0.1774]],
│           
│                   [[-0.1960,  0.2083,  1.5424],
│                    [-0.1976, -1.3767,  0.1774]],
│           
│                   [[-0.1960,  0.2083,  1.5424],
│                    [-0.1976, -1.3767,  0.1774]],
│           
│                   [[-0.1960,  0.2083,  1.5424],
│                    [-0.1976, -1.3767,  0.1774]]])
└── 'x' --> <FastTreeValue 0x7ff98115c730>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

46.7 µs ± 614 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff9800ec280>
├── 'a' --> tensor([[-0.1960,  0.2083,  1.5424],
│                   [-0.1976, -1.3767,  0.1774],
│                   [-0.1960,  0.2083,  1.5424],
│                   [-0.1976, -1.3767,  0.1774],
│                   [-0.1960,  0.2083,  1.5424],
│                   [-0.1976, -1.3767,  0.1774],
│                   [-0.1960,  0.2083,  1.5424],
│                   [-0.1976, -1.3767,  0.1774],
│                   [-0.1960,  0.2083,  1.5424],
│                   [-0.1976, -1.3767,  0.1774],
│                   [-0.1960,  0.2083,  1.5424],
│                   [-0.1976, -1.3767,  0.1774],
│                   [-0.1960,  0.2083,  1.5424],
│                   [-0.1976, -1.3767,  0.1774],
│                   [-0.1960,  0.2083,  1.5424],
│                   [-0.1976, -1.3767,  0.1774]])
└── 'x' --> <FastTreeValue 0x7ff9800ec130>
    └── 'c' --> tensor([[ 0.9793,  0.3693, -0.3448,  0.7939],
                        [-0.7109, -0.5054,  0.7105, -0.4517],
                 

In [23]:
%timeit t_cat(trees)

42.1 µs ± 484 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

84.4 µs ± 2.74 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.9793,  0.3693, -0.3448,  0.7939],
                       [-0.7109, -0.5054,  0.7105, -0.4517],
                       [-1.5664, -1.1374,  0.7007,  0.3352]],
              
                      [[ 0.9793,  0.3693, -0.3448,  0.7939],
                       [-0.7109, -0.5054,  0.7105, -0.4517],
                       [-1.5664, -1.1374,  0.7007,  0.3352]],
              
                      [[ 0.9793,  0.3693, -0.3448,  0.7939],
                       [-0.7109, -0.5054,  0.7105, -0.4517],
                       [-1.5664, -1.1374,  0.7007,  0.3352]],
              
                      [[ 0.9793,  0.3693, -0.3448,  0.7939],
                       [-0.7109, -0.5054,  0.7105, -0.4517],
                       [-1.5664, -1.1374,  0.7007,  0.3352]],
              
                      [[ 0.9793,  0.3693, -0.3448,  0.7939],
                       [-0.7109, -0.5054,  0.7105, -0.4517],
                       [-1.5664, -1.1374,  0.7007,  0.3352]],

In [26]:
%timeit Batch.stack(batches)

106 µs ± 974 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.9793,  0.3693, -0.3448,  0.7939],
                      [-0.7109, -0.5054,  0.7105, -0.4517],
                      [-1.5664, -1.1374,  0.7007,  0.3352],
                      [ 0.9793,  0.3693, -0.3448,  0.7939],
                      [-0.7109, -0.5054,  0.7105, -0.4517],
                      [-1.5664, -1.1374,  0.7007,  0.3352],
                      [ 0.9793,  0.3693, -0.3448,  0.7939],
                      [-0.7109, -0.5054,  0.7105, -0.4517],
                      [-1.5664, -1.1374,  0.7007,  0.3352],
                      [ 0.9793,  0.3693, -0.3448,  0.7939],
                      [-0.7109, -0.5054,  0.7105, -0.4517],
                      [-1.5664, -1.1374,  0.7007,  0.3352],
                      [ 0.9793,  0.3693, -0.3448,  0.7939],
                      [-0.7109, -0.5054,  0.7105, -0.4517],
                      [-1.5664, -1.1374,  0.7007,  0.3352],
                      [ 0.9793,  0.3693, -0.3448,  0.7939],
                   

In [28]:
%timeit Batch.cat(batches)

188 µs ± 1.86 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

609 µs ± 12 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
